<a href="https://colab.research.google.com/github/Sujal-gulia/Pest-and-Disease-Detection/blob/main/llava-pest-finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
output_folder = '/content/drive/MyDrive/dataset'
image_folder = '/content/drive/MyDrive/dataset/images'
train_json_path = '/content/drive/MyDrive/dataset/train.json'
val_json_path = '/content/drive/MyDrive/dataset/val.json'

# Update paths in fine-tuning script
data_path = "/content/drive/MyDrive/dataset/train.json"  # For training data
validation_data_path = "/content/drive/MyDrive/dataset/train.json"  # For validation data

In [ ]:
!git clone https://github.com/haotian-liu/LLaVA.git
!cd LLaVA && pip install --upgrade pip && pip install -e .

In [ ]:
!cd LLaVA && pip install -e ".[train]"
!pip install flash-attn --no-build-isolation

In [ ]:
!pip install deepspeed

In [ ]:
!pip install wandb

In [ ]:
import wandb

wandb.login()

In [ ]:
!deepspeed LLaVA/llava/train/train_mem.py \
    --lora_enable True --lora_r 128 --lora_alpha 256 --mm_projector_lr 2e-5 \
    --deepspeed LLaVA/scripts/zero3.json \
    --model_name_or_path liuhaotian/llava-v1.5-13b \
    --version v1 \
    --data_path ./dataset/train/dataset.json \
    --image_folder ./dataset/images \
    --vision_tower openai/clip-vit-large-patch14-336 \
    --mm_projector_type mlp2x_gelu \
    --mm_vision_select_layer -2 \
    --mm_use_im_start_end False \
    --mm_use_im_patch_token False \
    --image_aspect_ratio pad \
    --group_by_modality_length True \
    --bf16 True \
    --output_dir ./checkpoints/llava-v1.5-13b-task-lora \
    --num_train_epochs 1 \
    --per_device_train_batch_size 16 \
    --per_device_eval_batch_size 4 \
    --gradient_accumulation_steps 1 \
    --evaluation_strategy "no" \
    --save_strategy "steps" \
    --save_steps 50000 \
    --save_total_limit 1 \
    --learning_rate 2e-4 \
    --weight_decay 0. \
    --warmup_ratio 0.03 \
    --lr_scheduler_type "cosine" \
    --logging_steps 1 \
    --tf32 True \
    --model_max_length 2048 \
    --gradient_checkpointing True \
    --dataloader_num_workers 4 \
    --lazy_preprocess True \
    --report_to wandb

In [ ]:
!python LLaVA/scripts/merge_lora_weights.py --model-path checkpoints/llava-v1.5-13b-task-lora --model-base liuhaotian/llava-v1.5-13b --save-model-path llava-ftmodel

In [ ]:
!wget -L https://raw.githubusercontent.com/brevdev/notebooks/main/assets/llava-deploy.sh


In [ ]:
!chmod +x llava-deploy.sh && ./llava-deploy.sh